In [ ]:
import json
import os
from library_dicom.dicom_processor.model.Series import Series 
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.csv_reader.MaskBuilder import MaskBuilder

from library_dicom.dicom_processor.tools.folders import *

import numpy as np
import SimpleITK as sitk

In [ ]:
#LIST FROM PARSE_FILTERED_DICOM_DATASET

In [ ]:
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_list_path_csv.json'

In [ ]:
data = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        for path in reader[info] :
            data.append(path)

In [ ]:
dataset = []
for i in range(0,len(data),5):
    subliste = []
    subliste.append(data[i])
    subliste.append(data[i+1])
    subliste.append(data[i+2])
    subliste.append(data[i+3])
    subliste.append(data[i+4])
    dataset.append(subliste)

In [ ]:
print(len(dataset))


In [ ]:
#GENERE LA LISTE DES JSON AVEC UN UNCONSTANT SPACING, SAVE IT AS JSON 

In [ ]:

from library_dicom.dicom_processor.tools.folders import *
liste_unconstant_z_spacing = []
for liste in dataset : 
    print(dataset.index(liste))
    for i in range(0,4,2) : 
        serie_objet = Series(liste[i])
        serie_objet.get_numpy_array()
        if serie_objet.get_z_spacing() == 'Unconstant Spacing' : 
            print(liste)
            liste_unconstant_z_spacing.append(liste)
            list_z_spacing = serie_objet.calculate_z_spacing()
            print(list_z_spacing)
            #liste_unconstant_z_spacing.append(list_z_spacing)


#save it as json

write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_problem_unconstant_spacing', liste_unconstant_z_spacing)

In [ ]:
from library_dicom.dicom_processor.tools.folders import *
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_problem_unconstant_spacing', liste_unconstant_z_spacing)

In [ ]:
#On enleve les series avec un unconstant spacing 
for serie in liste_unconstant_z_spacing : 
    dataset.remove(serie)

In [ ]:
#SAVE AS JSON THE FINAL LIST WITHOUT UNCONSTANT SPACING ERROR

In [ ]:
from library_dicom.dicom_processor.tools.folders import *
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_final_list_dataset', dataset)


In [ ]:
print(len(dataset)) #589

In [ ]:
nifti_directory = '/media/salim/DD 2To/AHL2011_NIFTI'

csv_directory = '/media/salim/DD 2To/AHL2011_CSV_DATA/AHL2011-CSV'

In [ ]:
#IF MASK IS CORRECT => GENERATE NIFTI 
#IF PHILIPS, OR MASK FALSE => PUT SERIE IN A NEW LISTE 

In [ ]:
untreated_liste = []
#CT et PT pas dans le même ordre suivant les séries 
#voir pour les ranger avant PT en premier puis CT 
#error = []

for serie in dataset : 
    try : 
        print(serie)
        print(dataset.index(serie))
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            manufacturer = serie_pt_objet.get_series_details()['series']['Manufacturer'] 
            if 'philips' in manufacturer.lower() :
                untreated_liste.append(serie)
            else : 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
                size_matrix = serie_pt_objet.get_size_matrix()
                mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
                mask_4D = mask_objet.mask_array
                if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
                    print("MASK CORRECT")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                    #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[2])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                    print("EXPORT NIFTI MASK")
                else : 
                    untreated_liste.append(serie)
        else : 
            serie_pt_objet = SeriesPT(serie[2])
            manufacturer = serie_pt_objet.get_series_details()['series']['Manufacturer'] 
            if 'philips' in manufacturer.lower() :
                untreated_liste.append(serie)
            else : 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
                size_matrix = serie_pt_objet.get_size_matrix()
                mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
                mask_4D = mask_objet.mask_array
                if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
                    print("MASK CORRECT")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                     #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[0])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                    print("EXPORT NIFTI MASK")
                else : 
                    untreated_liste.append(serie)
                   


    except Exception as err : 
        print(err)
        #error.append(serie)

        


In [ ]:
len(untreated_liste) #392 

In [ ]:
#SAVE THE SERIE WITH FALSE MASK 
from library_dicom.dicom_processor.tools.folders import *
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_serie_with_false_mask', untreated_liste)
print(len(untreated_liste))

In [ ]:
#SEARCH SERIE WITH ERROR ET SAVE IT AS JSON 

In [ ]:
#IMPORTER JSON LISTE SANS UNCONSTANT SPACING POUR TROUVER LES SERIES QUI GENERE DES ERREURS
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_final_list_dataset.json'
data = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        for path in reader[info] :
            data.append(path)


dataset = []
for i in range(0,len(data),5):
    subliste = []
    subliste.append(data[i])
    subliste.append(data[i+1])
    subliste.append(data[i+2])
    subliste.append(data[i+3])
    subliste.append(data[i+4])
    dataset.append(subliste)

print(len(dataset)) #589

In [ ]:
#sauver les séries qui font des erreurs dans un autre json 
serie_with_error = []
for serie in dataset : 
    try : 
        #print(serie)
        print(dataset.index(serie))
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array

        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array

    except Exception as err : 
        print(serie)
        print(err)
        serie_with_error.append(serie)

from library_dicom.dicom_processor.tools.folders import *
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_serie_with_error', serie_with_error)
#On traitera après 

In [ ]:
#IMPORT SERIES WITH FALSE MASK,
#CHECK FLIP AXE Z + SMALL ERROR ON ELLIPSE 

In [ ]:
#importer la liste untreated_serie pour la suite : 
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_serie_with_false_mask.json'
data = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        for path in reader[info] :
            data.append(path)


untreated_liste = []
for i in range(0,len(data),5):
    subliste = []
    subliste.append(data[i])
    subliste.append(data[i+1])
    subliste.append(data[i+2])
    subliste.append(data[i+3])
    subliste.append(data[i+4])
    untreated_liste.append(subliste)

print(len(untreated_liste)) #392

In [ ]:
#sur la untreated_liste

#cad la ou le mask était en FALSE
#regarder si le flip de mask est good, si oui => generer nifti 
#regarder si les petites erreurs en mean/sd ne sont que sur les ellipses, si oui -> generer nifti 
untreated_liste_2 = []
error = []
for serie in untreated_liste : 
    try : 
        if serie[1] == 'PT' : 
           serie_pt_objet = SeriesPT(serie[0]) 
            manufacturer = serie_pt_objet.get_series_details()['series']['Manufacturer'] 
            #si philips on retraitera plus tard 
            if 'philips' in manufacturer.lower() :
                untreated_liste.append(serie)
            else : 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
                size_matrix = serie_pt_objet.get_size_matrix()
                mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
                mask_4D = mask_objet.mask_array
                #ON FLIP LES Z DE CHAQUE CHANNEL 3D
                mask_objet.flip_z(mask_4D)
                if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct APRES LE FLIP 
                    print("MASK CORRECT APRES FLIP")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                    #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[2])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                    print("EXPORT NIFTI MASK")

                else : #SI MASQUE TOUJOURS FAUX 
                    if mask_objet.ecart_suv_max(nifti_array) != [] :
                        #si l'erreur est sur les SUV MAX, regarder après 
                        untreated_liste_2.append(serie)
                    else : 
                        result_mean = mask_objet.ecart_suv_mean(nifti_array)
                        result_sd = mask_objet.ecart_SD(nifti_array)
                        if (('POLYGON' not in result_mean) and ('POLYGON' not in result_sd)) : 
                            #si erreur sur les sd et mean QUE sur les ellipses, on tolère 
                            print("SMALL ERRORS ON ELLIPSE, MASK CORRECT")
                            #generation nifti PT
                            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                            print("EXPORT NIFTI PT")
                            #generation nifti CT
                            serie_ct_objet = SeriesCT(serie[2])
                            serie_ct_objet.get_numpy_array()
                            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                            print("EXPORT NIFTI CT")
                            #generation nifti mask
                            filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                            print("EXPORT NIFTI MASK")
                        else : #sinon problème sur les MAX A CHECK
                            untreated_liste_2.append(serie)

        else : 
            serie_pt_objet = SeriesPT(serie[2]) 
            manufacturer = serie_pt_objet.get_series_details()['series']['Manufacturer'] 
            #si philips on retraitera plus tard 
            if 'philips' in manufacturer.lower() :
                untreated_liste.append(serie)
            else : 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
                size_matrix = serie_pt_objet.get_size_matrix()
                mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
                mask_4D = mask_objet.mask_array
                #ON FLIP LES Z DE CHAQUE CHANNEL 3D
                mask_objet.flip_z(mask_4D)
                if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct APRES LE FLIP 
                    print("MASK CORRECT APRES FLIP")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                    #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[0])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                    print("EXPORT NIFTI MASK")

                else : #SI MASQUE TOUJOURS FAUX 
                    if mask_objet.ecart_suv_max(nifti_array) != [] :
                        untreated_liste_2.append(serie)
                    else : 
                        result_mean = mask_objet.ecart_suv_mean(nifti_array)
                        result_sd = mask_objet.ecart_SD(nifti_array)
                        if (('POLYGON' not in result_mean) and ('POLYGON' not in result_sd)) : 
                            #si erreur sur les sd et mean QUE sur les ellipses, on tolère 
                            print("SMALL ERRORS ON ELLIPSE, MASK CORRECT")
                            #generation nifti PT
                            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                            print("EXPORT NIFTI PT")
                            #generation nifti CT
                            serie_ct_objet = SeriesCT(serie[0])
                            serie_ct_objet.get_numpy_array()
                            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                            print("EXPORT NIFTI CT")
                            #generation nifti mask
                            filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                            print("EXPORT NIFTI MASK")
                        else : #SINON PROBLM SUR LES MAX A CHECK 
                            untreated_liste_2.append(serie)


        except Exception as err : 
        print(err)
        print(serie)
        error.append(serie)




In [ ]:
print(len(untreated_liste_2))

In [ ]:
from library_dicom.dicom_processor.tools.folders import *
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_serie_untreated_liste_2', untreated_liste_2)


In [ ]:
#regarder les erreurs de max autres que philips 
#erreur sur certaines séries de NON RADIOPHARMACEUTICALS TAGS lié a philips ? 
#regarder les erreurs philips 
#problème de datetime dans certaines séries : 
        #Error generating result array time data '' does not match format '%Y%m%d%H%M%S'

In [ ]:
#philips -> MIP MASK 

In [ ]:
#error 
['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101061021/1.3.12.2.1107.5.1.4.45527.30000013030813185823400000037/1.3.12.2.1107.5.1.4.45527.30000013030814390209300004773/1.3.12.2.1107.5.1.4.45527.30000013030814390209300004773', 'CT', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101061021/1.3.12.2.1107.5.1.4.45527.30000013030813185823400000037/1.3.12.2.1107.5.1.4.45527.30000013030813205701500008501/1.3.12.2.1107.5.1.4.45527.30000013030813205701500008501', 'PT', 'pez_ol_mar 8_2013.csv']
449
index 170 is out of bounds for axis 0 with size 168

In [ ]:
['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101241004/2.16.840.1.113669.632.20.540001.10003028844/1.3.12.2.1107.5.1.4.44977.30000012032807144505000000000/1.3.12.2.1107.5.1.4.44977.30000012032807144505000000000', 'CT', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101241004/2.16.840.1.113669.632.20.540001.10003028844/1.3.12.2.1107.5.1.4.44977.30000012032806035646000000130/1.3.12.2.1107.5.1.4.44977.30000012032806035646000000130', 'PT', 'lau_ph_mar 28_2012-EV.csv']
542
float division by zero

['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101241007/2.16.840.1.113669.632.20.540001.10003053100/1.3.12.2.1107.5.1.4.44977.30000012041606392938900006833/1.3.12.2.1107.5.1.4.44977.30000012041606392938900006833', 'PT', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101241007/2.16.840.1.113669.632.20.540001.10003053100/1.3.12.2.1107.5.1.4.44977.30000012041608455152500003286/1.3.12.2.1107.5.1.4.44977.30000012041608455152500003286', 'CT', 'huo_fa_apr 16_2012-EV.csv']
544
float division by zero


['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101241012/2.16.840.1.113669.632.20.540001.10003399107/1.3.12.2.1107.5.1.4.44977.30000012123107075507500000126/1.3.12.2.1107.5.1.4.44977.30000012123107075507500000126', 'PT', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101241012/2.16.840.1.113669.632.20.540001.10003399107/1.3.12.2.1107.5.1.4.44977.30000012123107022041600000113/1.3.12.2.1107.5.1.4.44977.30000012123107022041600000113', 'CT', 'lau_to_dec 31_2012-EV.csv']
549
float division by zero